# T5

## Train

### Local

In [ ]:
from src.train.t5 import fit
from src.datasets import CFGenerativeDataset
from torch.utils.data import DataLoader, Subset
from transformers import T5ForConditionalGeneration

In [ ]:
BATCH_SIZE=16
EPOCHS=100
PATIENCE=10
SAVE_DIR="."
MODEL_NAME="t5-small"

In [ ]:
train_ds = CFGenerativeDataset("./configs/datasets/snli_1.0_contra.yaml", "./datasets/snli_1.0_contra", split="train")
val_ds = CFGenerativeDataset("./configs/datasets/snli_1.0_contra.yaml", "./datasets/snli_1.0_contra", split="val")

subset_indices = list(range(100))
train_ds = Subset(train_ds, subset_indices)
val_ds = Subset(val_ds, subset_indices)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

model=T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
fit(
    train_dl,
    val_dl,
    model,
    epochs= 2,
    patience= 10,
    save_dir= "models/t5-model",
)

### Sagemaker

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput

def train()->None:
    estimator = PyTorch(
        entry_point=f"sagemaker_t5.py",
        role="arn:aws:iam::065257926712:role/SagemakerRole",
        framework_version="2.0",
        py_version="py310",
        source_dir="src",
        output_path=f"s3://sliit-xai/training-jobs/results",
        code_location=f"s3://sliit-xai/training-jobs/code",
        instance_count=1,
        instance_type="ml.g4dn.xlarge",
        max_run=5 * 24 * 60 * 60
    )
    # Setting the input channels for tuning job
    s3_input_train = TrainingInput(s3_data="s3://sliit-xai/datasets/snli_1.0_contra/", s3_data_type="S3Prefix")

    # Start job
    estimator.fit(inputs={"train": s3_input_train})

train()

## Usage

In [ ]:
from src.cf_generators import T5Generator

cf_gen = T5Generator("./configs/models/t5-cf-generator.yaml", "./models/t5-cf-generator", download=True)
review = "\"Ice Age\" is an animated masterpiece that captivates both young and old audiences alike. The film's heartwarming and humorous storyline follows a mismatched group of prehistoric creatures on an epic adventure, which is filled with laughter, action, and valuable life lessons. The endearing characters, including Manny the mammoth, Sid the sloth, and Diego the saber-toothed tiger, effortlessly steal our hearts with their lovable quirks and undeniable chemistry. The animation is visually stunning, with breathtaking ice-capped landscapes and attention to detail that immerses viewers in a prehistoric wonderland. The movie's witty dialogue, clever jokes, and hilarious antics ensure that every moment is a joy to watch. Beyond the entertainment, \"Ice Age\" touches on themes of friendship, acceptance, and the importance of family, making it a truly heartwarming experience. This timeless classic stands the test of time, and its charm remains undiminished, making it a must-watch for anyone seeking an enchanting and delightful cinematic experience."
sentence_count = 4
contrads = cf_gen(review, sentence_count)

print("\n".join(contrads))

# WordFlippingGenerator

In [ ]:
from src.cf_generators import WordFlippingGenerator

review = "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."
sentence_count = 4

config_path = "./configs/models/wf-cf-generator.yaml"
wf = WordFlippingGenerator(config_path)
contrads = wf(review, sentence_count)
print("\n".join(contrads))

In [ ]:
wf.describe_tags()